In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf


def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

def resize_images(images, target_size=(224, 224)):
    """
    Resize images to the target size.
    """
    resized_images = [tf.image.resize(image, target_size).numpy() for image in images]
    return np.array(resized_images)


def calculating_iou(true_mask, pred_mask):
    arr_iou=[]
    for i in range(len(true_mask)):
        intersection = np.logical_and(true_mask[i],pred_mask[i])
        union = np.logical_or(true_mask[i], pred_mask[i])
        iou_score = np.sum(intersection) / np.sum(union)
        arr_iou.append(iou_score)
    arr_iou=np.array(arr_iou)
    arr_iou= np.nan_to_num(arr_iou, copy=True, nan=1.0)
    mean_iou=arr_iou.mean()
    return mean_iou


2024-09-18 15:07:37.413342: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-18 15:07:37.413389: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-18 15:07:37.414967: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-18 15:07:37.423315: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
test_image = np.load("../../numpy_arr_data/test_img.npy")
mask_image = np.load("../../numpy_arr_data/test_vegetation_mask_gray_img.npy")
mask_image = mask_image[..., np.newaxis]
test_image = resize_images(test_image)
mask_image = resize_images(mask_image)

print(f"Shape is {test_image.shape} {mask_image.shape}")

2024-09-18 15:07:40.312511: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-18 15:07:40.323615: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-18 15:07:40.327054: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Shape is (500, 224, 224, 3) (500, 224, 224, 1)


In [12]:
model = tf.keras.models.load_model("../model/mobilenet_model/dynamic/mobilenet_model_224_70_dynamiclr.keras")
predicted = model.predict(test_image, verbose=1)

pred_mask = []
for mask in predicted:
    mask = cv2.medianBlur(mask, 5)
    mask = rgb2gray(mask)
    mask[mask<0.5]=0
    mask[mask>=0.5]=1
    pred_mask.append(mask) 

mask_image = np.squeeze(mask_image)
mask_image = np.array(mask_image)
pred_mask = np.array(pred_mask)

print(calculating_iou(mask_image, pred_mask))

16/16 [==============================] - 2s 93ms/step
0.33429118812622205
